In [ ]:
import os
import cv2
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
c_dir = r'/kaggle/input/cat-and-dog/training_set/training_set'

In [ ]:
cat = cv2.imread(os.path.join(c_dir, 'cats/cat.788.jpg'))
cat = cv2.cvtColor(cat, cv2.COLOR_BGR2RGB)
plt.imshow(cat)

In [ ]:
# Getting random images from the train folder.
category = ['dogs','cats']
for i in category:
    path = os.path.join(c_dir,i)
    dir_path = os.listdir(path)
    random_pick = random.choices(dir_path)
    
    for j in random_pick:
        image = cv2.imread(os.path.join(path, j))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image)
        plt.xticks([])
        plt.yticks([])
    plt.show()

## Data Augmentation

In [ ]:
# Modifying Images:
train_set = ImageDataGenerator(rescale=1.0/255, 
                               shear_range=0.2, 
                               zoom_range=0.3, 
                               horizontal_flip=True,
                              vertical_flip=True,
                              brightness_range=(0.2,1.0),
                              rotation_range=30).flow_from_directory(directory= c_dir, 
                                                                         target_size=(256,256),
                                                                         batch_size=32, 
                                                                         class_mode='binary')

In [ ]:
# Just moulding the size of the image of test data.
test_set = ImageDataGenerator(rescale=1.0/255).flow_from_directory(directory= c_dir,
                                                                  target_size=(256,256), 
                                                                  class_mode= 'binary', 
                                                                  batch_size=32)

In [ ]:
# Checkpoint for the model to stop training if val_loss stop decresing after 3 epochs.
early_stops = EarlyStopping(monitor='val_loss', mode='min',patience=5)

# Model needs to be saved or else you will have to run model again and again which is time costly. 
# --So, Saving the model does
# helps in saving the best epoch of the trained model by considering val_loss(or can be accuracy) minimum.
callbacks_save = ModelCheckpoint('cat-dog classification model.hdf5', monitor='val_loss', mode='min', 
                                 save_best_only=True)

In [ ]:
cnn = Sequential()

cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu',input_shape=[256,256,3]))#adding input_size based on 
                                                                             #data-augmentation done earlier.
cnn.add(MaxPool2D(pool_size=3, strides=2))
#cnn.add(Dropout(0.1)) # adding a Dropout layer.

cnn.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
cnn.add(MaxPool2D(pool_size=2,strides=2))

cnn.add(Flatten())
cnn.add(Dense(1,activation='sigmoid')) # Sigmoid function because its best for binary classification.
cnn.compile(optimizer= 'Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
cnn.fit(x= train_set, validation_data=test_set, epochs=20, callbacks= [early_stops, callbacks_save])

In [ ]:
loss = cnn.history.history['loss']
accuracy = cnn.history.history['accuracy']
val_loss = cnn.history.history['val_loss']
val_accuracy = cnn.history.history['val_accuracy']

In [ ]:
plt.plot(loss,color='b',label='Training loss')
plt.plot(val_loss, color='r', label='validation loss')
plt.legend()
plt.title('Training and Validation loss')
plt.show()

In [ ]:
plt.plot(accuracy, color='b', label='Training accuracy')
plt.plot(val_accuracy, color='r', label='Validation accuracy')
plt.legend()
plt.title('Training and validation accuracy')
plt.show()

In [ ]:
model = load_model('cat-dog classification model.hdf5') # Loading model.

In [ ]:
c_dir

In [ ]:
# Importing random image from cat folder:

test_dir = r'/kaggle/input/cat-and-dog/training_set/training_set/cats'
path = os.listdir(test_dir) # listing it directory so random.choice() function can be applied.
random_pick = random.choice(path)

image = cv2.imread(os.path.join(test_dir, random_pick))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

In [ ]:
# Rescaling the image as per model which is(128,128,3)
image_test = cv2.resize(image, dsize=(256,256))
image_test = np.resize(image_test, (1,256,256,3))

prediction = model.predict(image_test) # prediction

print('prediction is', prediction, 'which is')
# Since the model predict in array zero or one so function define zero as cat and one as dog.
for i in prediction:
    if i==1:
        print('dog')
    else:
        print('cat')